# We will be computing surface EKE here

Import all the stuff we will need

In [79]:
  %matplotlib inline

  import matplotlib
  import numpy as np
  import matplotlib.pyplot as plt
  from matplotlib.colors import BoundaryNorm
  from matplotlib.ticker import MaxNLocator
  from netCDF4 import Dataset

  import numpy as np
  import Nio,Ngl
  import os,sys
  import datetime


Here are some things that will be sensitive to future uses of this script

In [80]:
  path_to_output = "/scratch/rhe/GoM/Data-Assimilation/jan2009b/"
  #-- open file
  f = Nio.open_file(path_to_output+"instance_0001/roms_posterior_0001_54939.nc","r")
  root = "/scratch/rhe/GoM/Data-Assimilation/jan2009b/"
  meanf = Nio.open_file("../ncs/mean_zeta_u_v_surface_full.nc","r")
  gridf = Nio.open_file(path_to_output+"instance_0001/useast_ini_0001.nc","r")
  lats=gridf.variables['lat_rho'][:,0]
  lons=gridf.variables['lon_rho'][0,:]
    
  latbound_s  = 20 #21
  latbound_n  = 30 #29
  lonbound_w  = -93 #-91
  lonbound_e  = -80 #-82

  instances = 33
  filesi=list(range(1,instances+1,1))
  filestringsi=[str(item).zfill(4) for item in filesi]
    
  wks_type = "png"
  wks = Ngl.open_wks(wks_type,"sfc_eke")
  Ngl.define_colormap(wks,"BlueYellowRed") 

  # Start date and end date
  start_date=datetime.datetime(2009,8,2)
  end_date=datetime.datetime(2009,8,31) # Fig 4.5-9

  #############################################################
  days_since=datetime.datetime(1858,11,17)
  start_date_ordinal=datetime.date.toordinal(start_date)
  end_date_ordinal=datetime.date.toordinal(end_date)
  days_since_ordinal=datetime.date.toordinal(days_since)
    
  start_date_index=start_date_ordinal-days_since_ordinal
  end_date_index=end_date_ordinal-days_since_ordinal
  # Optional sanity check
  #start_date_string=datetime.date.fromordinal(days_since_ordinal+start_date_index)
  nfiles=int((end_date_index-start_date_index+1)/2)
  filesi=list(range(start_date_index,end_date_index,2))
  filestringst=[str(item).zfill(4) for item in filesi]

Figure out date indices to loop over

In [81]:
  # Optional sanity check
  #start_date_string=datetime.date.fromordinal(days_since_ordinal+start_date_index)
  nfiles=int((end_date_index-start_date_index+1)/2)
  filesi=list(range(start_date_index,end_date_index,2))
  filestringst=[str(item).zfill(4) for item in filesi]
  mres                     = Ngl.Resources()
  mres.mpLimitMode           = "LatLon"   # Specify area of map
  mres.mpMaxLatF             = latbound_n       # to zoom in on.
  mres.mpMinLatF             = latbound_s
  mres.mpMaxLonF             = lonbound_e        # to zoom in on.
  mres.mpMinLonF             = lonbound_w
  mres.sfXArray             = lons         # Area of map to overlay 
  mres.sfYArray             = lats        # contours on.
  mres.mpProjection        = "Mercator"    # choose projection
  mres.tfDoNDCOverlay      = True                  # native grid, no transform
  mres.nglDraw             = False                 # don't draw yet
  mres.nglFrame            = False                 # don't advance frame yet
  mres.pmTickMarkDisplayMode = "Always"            # turn on tickmarks    

  res = Ngl.Resources()
  for t in dir(mres):
   if (t[0:2] == 'cn' or t[0:2] == 'sf' or t[0:3] == 'ngl'):
    setattr(res,t,getattr(mres,t))

  res.mpLimitMode           = "LatLon"   # Specify area of map

  res.sfXArray             = lons         # Area of map to overlay 
  res.sfYArray             = lats         # contours on.
  #plot_base = Ngl.map(wks,mres)
  #res.nglSpreadColorStart  = 99
  #res.nglSpreadColorEnd    = 2
  res.cnFillOn = True
  res.mpLimitMode           = "LatLon"   # Specify area of map
  res.mpMaxLatF             = latbound_n       # to zoom in on.
  res.mpMinLatF             = latbound_s
  res.mpMaxLonF             = lonbound_e        # to zoom in on.
  res.mpMinLonF             = lonbound_w  
  res.cnLevelSelectionMode = "ExplicitLevels" # explicit contour levels
  #res.cnLevels             = 0.17             # which level(s) to plot
  res.cnInfoLabelOn        = False            # no info label
  res.cnLineLabelsOn       = False            # no line labels
  res.cnLineThicknessF     = 4                # thickness of contour lines
  res.cnLineColor = -1
  res.pmLabelBarDisplayMode = "Never"
  res.cnLevelSelectionMode = "ManualLevels"  # Select contour levels.
  res.cnMinLevelValF       = 0.
  res.cnMaxLevelValF       = 1.
  res.cnLevelSpacingF      = 0.1
  res.mpGridLineColor = -1
  plot = []

  panelres = Ngl.Resources()
  panelres.pmLabelBarOrthogonalPosF = 0.02
  panelres.nglPanelLabelBar                 = True     # Turn on panel labelbar
  panelres.nglPanelLabelBarLabelFontHeightF = 0.015    # Labelbar font height
  panelres.nglPanelLabelBarHeightF          = 0.1350   # Height of labelbar
  panelres.nglPanelLabelBarWidthF           = 0.700    # Width of labelbar
  panelres.lbLabelFont                      = "helvetica-bold" # Labelbar font
  panelres.nglPanelTop                      = 0.925
  panelres.nclPanelBottom                   = 0.07
  
  
  paneli=list(range(1,instances+3,1))
  paneli[33]="pr"
  paneli[34]="po"
  panelstrings=[str(item) for item in paneli]

  panelres.nglPanelFigureStrings            =  panelstrings
  panelres.nglPanelFigureStringsJust        = "BottomRight"  
  panelres.nglPanelFigureStringsFontHeightF = 0.01
  nplots=35

  textres               = Ngl.Resources()

### We will use the ensemble mean rather than any individual member

In [93]:
  import numpy.ma as ma
  umean = meanf.variables['u'][:,:]
  vmean = meanf.variables['v'][:,:]
  zmean = meanf.variables['zeta'][:,:] 
    
  umean[umean>10] = 0
  vmean[vmean>10] = 0
    
  umean[umean<-10] = 0
  vmean[vmean<-10] = 0
    
  umean[umean == float('+inf') ] = 0
  vmean[vmean == float('+inf') ] = 0
    
  umean[umean == float('-inf') ] = 0
  vmean[vmean == float('-inf') ] = 0  
  njni=umean.shape  
  nj=njni[0]
  ni=njni[1]
  
  for t in range(0,1):
    plot = []
    date_obj=datetime.date.fromordinal(int(filestringst[t])+days_since_ordinal)        
    date_str=date_obj.strftime('%b %e, %Y')
    print(date_str)
    f = Nio.open_file(path_to_output+"analysis_mean."+str(filesi[t])+".nc","r")
    u=f.variables['u']['i0 35 : :']
    v=f.variables['v']['i0 35 : :']
    u[u<-10] = 0
    v[v<-10] = 0
    u[u>10] = 0
    v[v>10] = 0
    u[u == float('+inf') ] = 0
    v[v == float('+inf') ] = 0
    u[u == float('-inf') ] = 0
    v[v == float('-inf') ] = 0
    dimsu  = u.shape
    dimXu   = dimsu[1]
    ur = np.empty(njni)

    ur[:,1:dimXu-1] = 0.5*(u[:,:dimXu-2] + u[:,1:dimXu-1])
    ur[:,0]=ur[:,1]
    ur[:,dimXu]=ur[:,dimXu-1]
    
    dimsv  = v.shape
    dimYv   = dimsv[0]

    vr = np.empty(njni)
    vr[1:dimYv-1,:] = 0.5*(v[:dimYv-2,:] + v[1:dimYv-1,:])
    vr[0,:]=vr[1,:]
    vr[dimYv,:]=vr[dimYv-1,:]
    
    # GET THE POSTERIOR FIRST
    upo_prime = ur[:,:]-umean[:,:]
    vpo_prime = vr[:,:]-vmean[:,:]
    upo_prime[np.abs(upo_prime)>10] = 0
    vpo_prime[np.abs(vpo_prime)>10] = 0
    ekepo=0.5*((upo_prime[:,:]**2)+(vpo_prime[:,:])**2)
    ekepr_mean = np.empty(njni)
    for i in range(0,instances):
      #print(root+"instance_"+filestringsi[i]+"/roms_his_"+filestringsi[i]+"_"+filestringst[t]+".nc")
      f = Nio.open_file(root+"instance_"+filestringsi[i]+"/roms_his_"+filestringsi[i]+"_"+filestringst[t]+".nc","r")

      u=f.variables['u']['i0 35 : :']
      v=f.variables['v']['i0 35 : :']

      ur = np.empty(njni)  
      ur[:,1:dimXu-1] = 0.5*(u[:,:dimXu-2] + u[:,1:dimXu-1])
      ur[:,0]=ur[:,1]
      ur[:,dimXu]=ur[:,dimXu-1]
        
      vr = np.empty(njni)
      vr[1:dimYv-1,:] = 0.5*(v[:dimYv-2,:] + v[1:dimYv-1,:])
      vr[0,:]=vr[1,:]
      vr[dimYv,:]=vr[dimYv-1,:]   
      # PLOT EACH MEMBER   
      u_prime=np.empty(njni)
      v_prime=np.empty(njni)
      u_prime = ur[:,:]-umean[:,:]
      v_prime == vr[:,:]-vmean[:,:]  
      u_prime[np.abs(u_prime)>100] = 0
      v_prime[np.abs(v_prime)>100] = 0
  
      #eke=np.empty(njni) # WHY DO I NEED THIS?!?! WHY DOES THE NEXT LINE ADD INSTEAD OF OVERWRITE?!
      eke=0.5*((u_prime[:,:]**2)+(v_prime[:,:]**2))
      eke[eke<-10] = 0
      eke[eke<-10] = 0
      eke[eke>10] = 0
      eke[eke>10] = 0
      #ma.masked_where(zmean>1000,eke)
      plot.append(Ngl.contour_map(wks,eke,res))

      # Add up prior mean EKE
      if i==0:
        ekepr_mean = eke[:,:]
      else:
        ekepr_mean[:,:] = ekepr_mean[:,:] + eke[:,:]
      #print(ekepr_mean.max())

      print(i,ekepr_mean.max())
        
    # COMPUTE PRIOR MEAN NOW
    
    ekepr_mean = ekepr_mean/instances

    #f = Nio.open_file(root+"analysis_mean."+filestringst[t]+".nc","r")
    #zpost = f.variables['zeta'][0,:,:]
    plot.append(Ngl.contour_map(wks,ekepr_mean,res))
    plot.append(Ngl.contour_map(wks,ekepo,res))
    
    textres.txFontHeightF = 0.025   # Size of title.
    #Ngl.draw_ndc_grid(wks)
    Ngl.text_ndc(wks,date_str,0.5,.83,textres)
    Ngl.panel(wks,plot[0:nplots],[5,7],panelres)       
   

Aug  2, 2009
0 9.992182941754727
1 19.90383338351674
2 29.805687087432666
3 39.60956624436143
4 49.19293154239075
5 58.812745771200305
6 68.42365271066365
7 77.85181095417693
8 87.41365517112322
9 96.85770952261976
10 106.46244518828607
11 116.15944029949588
12 125.69452944661184
13 135.20361936132966
14 145.0038769161783
15 154.9755684331507
16 164.63324374045402
17 174.0726532202357
18 183.55978928560185
19 193.371353031788
20 193.371353031788
21 203.18841920200794
22 212.89188630631622
23 222.75968077522168
24 232.51615125774362
25 242.14925036376093
26 251.91863243789265
27 261.88984575565314
28 271.4446605886337
29 280.9822295426357
30 290.7777216740342
31 300.37488736901025
32 310.11019903537834
